In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix  # Импортируем confusion_matrix

# Загрузка данных
df = pd.read_csv('WineDataset.csv')

# Обработка отсутствующих значений
if df.isnull().sum().sum() > 0:
    df = df.fillna(df.mean()) 

# Отделение от целевой переменной
X = df.drop('Wine', axis=1).values  
y = df['Wine'].values 

# Нормализация
X = (X - np.min(X, axis=0)) / (np.max(X, axis=0) - np.min(X, axis=0))

# Обучающая и тестовая выборки
train_size = int(0.8 * X.shape[0])
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# KNN класс
class KNN:
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)
    
    def _predict(self, x):
        distances = np.linalg.norm(self.X_train - x, axis=1)
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

# Случайный выбор 5 признаков
np.random.seed(42)
random_features = np.random.choice(df.columns[:-1], 5, replace=False)
random_indices = [df.columns.get_loc(feature) for feature in random_features]

X_train_random = X[:train_size, random_indices]
X_test_random = X[train_size:, random_indices]

# Модель 1: k-NN с случайно выбранными признаками
knn_model_1 = KNN(k=3)
knn_model_1.fit(X_train_random, y_train)
y_pred_1 = knn_model_1.predict(X_test_random)

# Оценка для случайных признаков с использованием sklearn
conf_matrix_model_1 = confusion_matrix(y_test, y_pred_1)
print(f"Матрица ошибок для модели с случайными признаками (k=3):\n{conf_matrix_model_1}")

# Фиксированный набор признаков
fixed_features = ['Alcohol', 'Malic Acid', 'Ash']
fixed_indices = [df.columns.get_loc(feature) for feature in fixed_features]

X_train_fixed = X[:train_size, fixed_indices]
X_test_fixed = X[train_size:, fixed_indices]

# Модель 2: k-NN с фиксированными признаками
knn_model_2 = KNN(k=3)
knn_model_2.fit(X_train_fixed, y_train)
y_pred_2 = knn_model_2.predict(X_test_fixed)

# Оценка для фиксированных признаков с использованием sklearn
conf_matrix_model_2 = confusion_matrix(y_test, y_pred_2)
print(f"Матрица ошибок для модели с фиксированными признаками (k=3):\n{conf_matrix_model_2}")

def evaluate_knn(k_values, X_train, X_test, y_train, y_test):
    for k in k_values:
        knn = KNN(k=k)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        
        # Использование sklearn для матрицы ошибок
        conf_matrix = confusion_matrix(y_test, y_pred)
        print(f"Матрица ошибок для k={k}:\n{conf_matrix}\n")

k_values = [1, 2, 3, 5, 10]

print("Оценка для модели с случайными признаками:")
evaluate_knn(k_values, X_train_random, X_test_random, y_train, y_test)

print("Оценка для модели с фиксированными признаками:")
evaluate_knn(k_values, X_train_fixed, X_test_fixed, y_train, y_test)


Матрица ошибок для модели с случайными признаками (k=3):
[[ 0  0]
 [ 2 34]]
Матрица ошибок для модели с фиксированными признаками (k=3):
[[ 0  0  0]
 [ 0  0  0]
 [20  6 10]]
Оценка для модели с случайными признаками:
Матрица ошибок для k=1:
[[ 0  0  0]
 [ 0  0  0]
 [ 2  2 32]]

Матрица ошибок для k=2:
[[ 0  0  0]
 [ 0  0  0]
 [ 2  4 30]]

Матрица ошибок для k=3:
[[ 0  0]
 [ 2 34]]

Матрица ошибок для k=5:
[[ 0  0  0]
 [ 0  0  0]
 [ 2  1 33]]

Матрица ошибок для k=10:
[[ 0  0  0]
 [ 0  0  0]
 [ 2  1 33]]

Оценка для модели с фиксированными признаками:
Матрица ошибок для k=1:
[[ 0  0  0]
 [ 0  0  0]
 [18  8 10]]

Матрица ошибок для k=2:
[[ 0  0  0]
 [ 0  0  0]
 [21  8  7]]

Матрица ошибок для k=3:
[[ 0  0  0]
 [ 0  0  0]
 [20  6 10]]

Матрица ошибок для k=5:
[[ 0  0  0]
 [ 0  0  0]
 [21 10  5]]

Матрица ошибок для k=10:
[[ 0  0  0]
 [ 0  0  0]
 [19  9  8]]

